# Evaluation

In [ ]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
hotel_rev = pd.read_csv('data/HotelRevHelpfulness.csv')
hotel_rev.pop("hotelId")   # get rid of ID feature
hotel_rev.head()

'reviewHelpfulness' is the class label. `1` is positive and `0` is negative.

In [ ]:
n = hotel_rev.shape[0]
np = hotel_rev['reviewHelpfulness'].sum()
nn = n - np
print('{} Samples'.format(n))
print('{} Positive \n{} Negative'.format(np,nn))

## Hold Out Validation

In [ ]:
y = hotel_rev.pop('reviewHelpfulness').values   #target classes
Xraw = hotel_rev.values 
# normalise the raw data
scaler = StandardScaler()
X = scaler.fit_transform(Xraw)
# split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=1/3)
print("X_test shape: ",X_test.shape)

In [ ]:
# train a NB model and predict for test data
mnb = GaussianNB()
hotel_rev_NB = mnb.fit(X_train, y_train)
y_dash = hotel_rev_NB.predict(X_test)

## Accuracy & Confusion Matrix
With the confusion matrix, rows are actual and columns are predicted.   
If 0 is negative and 1 is positive `C(0,0)` is TN and `C(1,1)` is TP.  
TN, FP  
FN, TP


In [ ]:
acc = accuracy_score(y_test, y_dash)
print("Accuracy: {0:.2f}".format(acc)) 
confusion = confusion_matrix(y_test, y_dash)
print("Confusion matrix:\n{}".format(confusion)) 

In [ ]:
report = classification_report(y_test, y_dash, target_names = ['Not Helpful','Helpful'])
print("Report:\n{}".format(report)) 

## Cross Validation
`cross_val_score` will run a k-fold cross validation.  


In [ ]:
from sklearn.model_selection import cross_val_score
mnb = GaussianNB()
scores = cross_val_score(mnb, X, y, cv=5)
print("5x CV Accuracy Naive Bayes: {0:.2f}".format(scores.mean()))  
scores   # print scores for each fold

In [ ]:
kNN = KNeighborsClassifier(n_neighbors=3)  
kNN_scores = cross_val_score(kNN, X, y, cv=5)
print("5x CV Accuracy kNNs: {0:.2f}".format(kNN_scores.mean())) 
kNN_scores


In [ ]:
dtree = DecisionTreeClassifier(criterion='entropy')
tree_scores = cross_val_score(dtree, X, y, cv=5)
print("5x CV Accuracy Trees: {0:.2f}".format(tree_scores.mean())) 
tree_scores

Which classifier is best?

## F1 Score
A balanced score that measures the harmonic mean between Precision and Recall.

In [ ]:
models = [mnb,kNN,dtree]    
folds = 5

In [ ]:
print('F1 Measure')
for m in models:
    f1_scores = cross_val_score(m, X, y, cv=folds, scoring = 'f1', n_jobs=-1, verbose = 5)
    print("{} x CV {:22} {:.2f}".format(folds, type(m).__name__, f1_scores.mean())) 

## Precision & Recall

In [ ]:
print('Precision')
for m in models:
    f1_scores = cross_val_score(m, X, y, cv=folds, scoring = 'precision', n_jobs=-1, verbose = 5)
    print("{} x CV {:22} {:.2f}".format(folds, type(m).__name__, f1_scores.mean())) 

In [ ]:
print('Recall')
for m in models:
    f1_scores = cross_val_score(m, X, y, cv=folds, scoring = 'recall', n_jobs=-1, verbose = 5)
    print("{} x CV {:22} {:.2f}".format(folds, type(m).__name__, f1_scores.mean())) 

Why are Recall figures different from Accuracy?

## ROC AUC analysis

In [ ]:
folds = 5
print('ROC Analysis')
for m in models:
    roc_scores = cross_val_score(m, X, y, cv=folds, scoring = 'roc_auc', n_jobs=-1)
    print("{} x CV {:22} {:.2f}".format(folds, type(m).__name__, roc_scores.mean())) 


## Calculating TP & FP with cross validation
The hold out validation shows that there are high FP and FN rates.   
Here we get a cross-validation estimate of these rates.  
To do this we have to make a scorer function to get the individual scores. 

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
def tp(y_true, y_pred): return confusion_matrix(y_true, y_pred)[1, 1]
def tn(y_true, y_pred): return confusion_matrix(y_true, y_pred)[0, 0]
def fp(y_true, y_pred): return confusion_matrix(y_true, y_pred)[0, 1]
def fn(y_true, y_pred): return confusion_matrix(y_true, y_pred)[1, 0]
scoring = {'tp' : make_scorer(tp), 'tn' : make_scorer(tn),
           'fp' : make_scorer(fp), 'fn' : make_scorer(fn)}

models = [mnb,kNN,dtree]

folds = 5
v = 0 #  use 1 or 0, try both

In [ ]:
for m in models:
    cv_results = cross_validate(m, X, y, cv= folds, scoring=scoring, return_train_score=False, 
                                    verbose = v, n_jobs = -1)
    fp_rate = cv_results['test_fp'].sum()/(cv_results['test_fp'].sum()+cv_results['test_tn'].sum())
    tp_rate = cv_results['test_tp'].sum()/(cv_results['test_tp'].sum()+cv_results['test_fn'].sum())
    fn_rate = cv_results['test_fn'].sum()/(cv_results['test_fn'].sum()+cv_results['test_tp'].sum())
    tn_rate = cv_results['test_tn'].sum()/(cv_results['test_tn'].sum()+cv_results['test_fp'].sum())
  
    print("{} x CV {:22} FP: {:.2f}  TP: {:.2f}  FN: {:.2f}  TN: {:.2f}  ".format(folds, type(m).__name__, fp_rate, tp_rate, fn_rate, tn_rate)) 


See how the different classifiers perform across the different measures




## Plot the confusion matrix

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils.multiclass import unique_labels
%matplotlib inline

def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    #classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plot_confusion_matrix(y_test, y_dash, classes=['Not Helpful','Helpful'],
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plot_confusion_matrix(y_test, y_dash, classes=['Not Helpful','Helpful'], normalize=True,
                      title='Normalized confusion matrix')

plt.show()